In [7]:
import os
import pandas as pd
import seaborn as sns

In [3]:

from mistralai.client import MistralClient

In [1]:
from mistralai.models.jobs import TrainingParameters

In [5]:
api_key = os.environ.get('MISTRAL_API_KEY')

In [83]:
client = MistralClient(api_key=api_key)

In [9]:
clinical_notes = pd.read_csv('../data/annotations/clinical_notes.csv')

In [54]:
annotations = pd.read_csv('../data/annotations/annotations.csv')

In [40]:
snomed_ct = pd.read_csv('~/BSC/code/hackbcn-2024/data/vocabularies/SnomedCT_Spanish_Edition/SnomedCT_SpanishRelease-es_PRODUCTION_20210430T120000Z/Full/Terminology/sct2_Description_SpanishExtensionFull-es_INT_20210430.txt', sep='\t')

In [41]:
snomed = snomed_ct.get(['conceptId', 'term']).rename(columns={'conceptId': 'code'})

In [42]:
snomed.code = snomed.code.astype('str')

In [43]:
snomed.to_csv('../data/snomed.csv', index=False)

In [47]:
snomed.query("code=='34095006'").drop_duplicates('code')

,code,term
97166,34095006,deshidratación


In [37]:
annotations

,filename,label,span_start,span_end,span_text,code
0,es-S0210-56912007000900007-3,ENFERMEDAD,164,166,DM,73211009
1,es-S0210-56912007000900007-3,ENFERMEDAD,362,376,deshidratación,34095006
2,es-S0210-56912007000900007-3,ENFERMEDAD,575,590,hiperamilasemia,275739007
3,es-S0210-56912007000900007-3,ENFERMEDAD,715,733,pancreatitis aguda,197456007
4,es-S0210-56912007000900007-3,ENFERMEDAD,1402,1459,formación polipoidea sésil situada junto al es...,88580009
...,...,...,...,...,...,...
26706,es-S2340-98942015000100005-1,PROCEDIMIENTO,1732,1795,quimioterapia con trabectedina y adriamicina l...,367336001
26707,es-S2340-98942015000100005-1,PROCEDIMIENTO,231,284,quimioterapia adyuvante con carboplatino y pac...,367336001
26708,es-S2340-98942015000100005-1,PROCEDIMIENTO,465,468,TAC,77477000
26709,es-S2340-98942015000100005-1,PROCEDIMIENTO,533,569,resección anterior de recto superior,172341002


In [49]:
snomed_es = snomed.drop_duplicates('code')

In [50]:
annotations.merge(snomed_es, on='code')

,filename,label,span_start,span_end,span_text,code,term
0,es-S0210-56912007000900007-3,ENFERMEDAD,164,166,DM,73211009,diabetes mellitus
1,es-S0210-56912007000900007-3,ENFERMEDAD,164,166,DM,73211009,diabetes sacarina
2,es-S0210-56912007000900007-3,ENFERMEDAD,164,166,DM,73211009,diabetes mellitus (trastorno)
3,es-S0210-56912007000900007-3,ENFERMEDAD,362,376,deshidratación,34095006,deshidratación
4,es-S0210-56912007000900007-3,ENFERMEDAD,362,376,deshidratación,34095006,síndrome de depleción de agua libre
...,...,...,...,...,...,...,...
94950,es-S2340-98942015000100005-1,PROCEDIMIENTO,533,569,resección anterior de recto superior,172341002,resección del músculo recto superior (procedim...
94951,es-S2340-98942015000100005-1,PROCEDIMIENTO,533,569,resección anterior de recto superior,172341002,resección del recto superior
94952,es-S2340-98942015000100005-1,PROCEDIMIENTO,533,569,resección anterior de recto superior,172341002,resección del músculo recto superior
94953,es-S2340-98942015000100005-1,PROCEDIMIENTO,574,620,biopsia intraoperatoria de una lesión hepática,274355008,biopsia quirúrgica de hígado (procedimiento)


In [14]:
filename='es-S2340-98942015000100005-1'

In [20]:
f'filename = "{filename}"'

'filename = "es-S2340-98942015000100005-1"'

In [60]:
import json

In [75]:
annotations[annotations['filename'] == filename][['label','term']]

,label,term
8720,SINTOMA,temperatura corporal normal
9220,SINTOMA,colección pancreática
9225,SINTOMA,colección pancreática
9226,SINTOMA,trastorno de compartimiento retroperitoneal
9369,SINTOMA,cultivo microbiológico positivo (hallazgo)
9399,SINTOMA,balance de líquidos de drenaje - hallazgo
9504,SINTOMA,deterioro general de la salud
10174,SINTOMA,fiebre
10752,SINTOMA,colección intrabdominal
11555,SINTOMA,fiebre


In [11]:
from tqdm import tqdm 

In [55]:
annotations = annotations.merge(snomed_es, on='code')

In [78]:
'| '.join([f"{row['label']} : {row['term']}" for i, row in annot.iterrows()])

'SINTOMA : temperatura corporal normal| SINTOMA : colección pancreática| SINTOMA : colección pancreática| SINTOMA : trastorno de compartimiento retroperitoneal| SINTOMA : cultivo microbiológico positivo (hallazgo)| SINTOMA : balance de líquidos de drenaje - hallazgo| SINTOMA : deterioro general de la salud| SINTOMA : fiebre| SINTOMA : colección intrabdominal| SINTOMA : fiebre'

In [79]:
# Process each unique filename
messages = []

for i, row in tqdm(clinical_notes.iterrows()):
    filename = row['filename']
    text = row['text']
    annot = annotations[annotations['filename'] == filename][['label', 'term']]
    
    query = json.dumps(annot.groupby('label').agg('; '.join).to_dict().get('term'))
    input = '| '.join([f"{row['label']} : {row['term']}" for i, row in annot.iterrows()])
    messages.append(
    {
        "messages": [
            {"role": "user", 
             "content": input
            },
            {"role": "assistant", 
             "content": text
            },
        ]
    }

    )



   



1000it [00:03, 315.03it/s]


In [72]:
len(messages)

1000

In [92]:
# Write data to a JSONL file
with open('training_file.jsonl', 'w') as file:
    for entry in messages[:900]:
        json.dump(entry, file)
        file.write('\n')


In [93]:
# Write data to a JSONL file
with open('test_file.jsonl', 'w') as file:
    for entry in messages[900:]:
        json.dump(entry, file)
        file.write('\n')

In [94]:
with open("training_file.jsonl", "rb") as f:
    training_data = client.files.create(file=("training_file.jsonl", f))    

In [95]:
with open("test_file.jsonl", "rb") as f:
    validation_data = client.files.create(file=("test_file.jsonl", f))  

In [96]:
created_job = client.jobs.create(
    model="open-mistral-7b",
    training_files=[training_data.id],
    validation_files=[validation_data.id],
    hyperparameters=TrainingParameters(
        training_steps=10,
        learning_rate=0.0001,
        )
)
created_job

Job(id='e98cba77-acf3-4e20-ab56-5f62ca77eb57', hyperparameters=TrainingParameters(training_steps=10, learning_rate=0.0001), fine_tuned_model=None, model='open-mistral-7b', status='QUEUED', job_type='FT', created_at=1719666118, modified_at=1719666118, training_files=['7f75f5d5-32c5-4478-912e-3462ef4d7e62'], validation_files=['d3fc2c99-b992-47cc-8091-79016d0dbcc6'], object='job', integrations=[])

In [98]:
retrieved_job = client.jobs.retrieve(created_job.id)
print(retrieved_job)

id='e98cba77-acf3-4e20-ab56-5f62ca77eb57' hyperparameters=TrainingParameters(training_steps=10, learning_rate=0.0001) fine_tuned_model='ft:open-mistral-7b:953eb039:20240629:e98cba77' model='open-mistral-7b' status='SUCCESS' job_type='FT' created_at=1719666118 modified_at=1719666204 training_files=['7f75f5d5-32c5-4478-912e-3462ef4d7e62'] validation_files=['d3fc2c99-b992-47cc-8091-79016d0dbcc6'] object='job' integrations=[] events=[Event(name='status-updated', data={'status': 'SUCCESS'}, created_at=1719666204), Event(name='status-updated', data={'status': 'RUNNING'}, created_at=1719666118), Event(name='status-updated', data={'status': 'QUEUED'}, created_at=1719666118)] checkpoints=[Checkpoint(metrics=Metric(train_loss=1.59191, valid_loss=1.607452, valid_mean_token_accuracy=3.047133), step_number=10, created_at=1719666191)] estimated_start_time=None


In [100]:
fine_tuned_model = retrieved_job.fine_tuned_model


In [101]:
fine_tuned_model

'ft:open-mistral-7b:953eb039:20240629:e98cba77'

In [112]:
from mistralai.models.chat_completion import ChatMessage

In [103]:
import random

In [105]:
test = random.choice(messages)

In [107]:
test['messages'][0]['content']

'ENFERMEDAD : herida de entrada de disparo de arma de fuego (trastorno)| ENFERMEDAD : herida de bala| ENFERMEDAD : hiperplasia de próstata| ENFERMEDAD : cuerpo extraño de una estructura corporal| ENFERMEDAD : fractura de la rama inferior del pubis| ENFERMEDAD : perforación del recto| ENFERMEDAD : lesión traumática Y/O atraumática| ENFERMEDAD : lesión traumática Y/O atraumática| ENFERMEDAD : perforación del recto| SINTOMA : ausencia de síntomas| SINTOMA : completamente consciente| SINTOMA : orientado (hallazgo)| SINTOMA : defensa abdominal| SINTOMA : fiebre| SINTOMA : escalofríos| SINTOMA : ausencia de síntomas| SINTOMA : función sexual normal| SINTOMA : hallazgo uretral| SINTOMA : retención de orina| SINTOMA : ecografía normal| PROCEDIMIENTO : procedimiento quirúrgico (procedimiento)| PROCEDIMIENTO : colonoscopia| PROCEDIMIENTO : endoscopia de la uretra (procedimiento)| PROCEDIMIENTO : colocación de sonda uretral (procedimiento)| PROCEDIMIENTO : procedimiento quirúrgico (procedimiento)

In [109]:
test['messages'][1]['content']

'Se trata de un paciente de 59 años que ingresa con una herida de arma de fuego con orificio de entrada en la nalga, de unos 4 milímetros de diámetro y sin orificio de salida. Además presenta otra herida de bala en el muslo derecho con orificio de entrada por detrás y salida por delante, sin afectación de estructuras importantes. El paciente está consciente, orientado y con una ligera defensa abdominal. En la radiografía simple de abdomen aparece una bala situada a la altura de la sínfisis del pubis, así como una fractura de la rama descendente derecha del mismo. El enfermo llega en retención, siendo imposible el sondaje, por lo que se pone un catéter suprapúbico. Después se efectúa una laparotomía, no encontrándose ninguna otra lesión, realizándose una colostomía izquierda. Al día siguiente se realiza una uretrografía, comprobándose que hay paso de contraste desde la uretra hasta el recto, lo que provoca fiebre alta con escalofríos que se mantiene durante unos días. Posteriormente, ci

In [ ]:


chat_response = client.chat(
    model=fine_tuned_model,
    messages=[ChatMessage(role='user', 
                          content='What is the best French cheese?'
                          )
                ]
)

In [ ]:
from mistralai.models.chat_completion import ChatMessage
fine_tuned_model = 'ft:open-mistral-7b:953eb039:20240629:e98cba77'
input = 'SINTOMA : temperatura corporal normal| SINTOMA : colección pancreática| SINTOMA : colección pancreática| SINTOMA : trastorno de compartimiento retroperitoneal| SINTOMA : cultivo microbiológico positivo (hallazgo)| SINTOMA : balance de líquidos de drenaje - hallazgo| SINTOMA : deterioro general de la salud| SINTOMA : fiebre| SINTOMA : colección intrabdominal| SINTOMA : fiebre'
def mistral_call(input):
    chat_response = client.chat(
    model=fine_tuned_model,
    messages=[ChatMessage(role='user', 
                          content=input
                          )
                ]
    )

    return chat_response.choices[0].message.content


In [135]:
input

'SINTOMA : temperatura corporal normal| SINTOMA : colección pancreática| SINTOMA : colección pancreática| SINTOMA : trastorno de compartimiento retroperitoneal| SINTOMA : cultivo microbiológico positivo (hallazgo)| SINTOMA : balance de líquidos de drenaje - hallazgo| SINTOMA : deterioro general de la salud| SINTOMA : fiebre| SINTOMA : colección intrabdominal| SINTOMA : fiebre'

In [ ]:
from pprint import pprint

In [136]:
def test():
    test = random.choice(messages)
    input = test['messages'][0]['content']
    expected = test['messages'][1]['content']
    chat_response = client.chat(
    model=fine_tuned_model,
    messages=[ChatMessage(role='user', 
                          content=input
                          )
                ]
    )

    response = chat_response.choices[0].message.content
    pprint(f'Original: {expected}')
    pprint(f'Model: {response}')

    return response


In [144]:
def explain(input):
    client = MistralClient(api_key=api_key)
    model = "mistral-large-latest"

    sys_message = """
        You are physician. You have been given the clinical record for a patient. Summarize the report so that it is easily understood for a non expert, avoiding difficult medical terms.
    """

    messages = [
        ChatMessage(role="system", content=sys_message),
        ChatMessage(role="user", content=input)
    ]
    chat_response = client.chat(
        model=model,
        messages=messages
    )
    return chat_response.choices[0].message.content



In [143]:
%%time
resp = test()

MistralException: Unexpected exception (ReadTimeout): The read operation timed out

In [145]:
expected


NameError: name 'expected' is not defined

In [142]:
%%time
exp = explain(resp)
pprint(f'Explanation: {exp}')

('Explanation: Un hombre de 53 años fue diagnosticado de mieloma de IgG y '
 'tratado con prednisona, melphalán y bortezomib. Presentó un tumor en la '
 'cámara anterior de ojo derecho, que fue diagnosticado como retinoblastoma '
 'plasmocitario. El paciente falleció tras radioterapia. El diagnóstico de '
 'autopsia fue de retinoblastoma plasmocitario, con fibrosis. En su paciente '
 'reciente, el diagnóstico de biopsia fue de tumor retinoblástico, pero en la '
 'autopsia se apreció una infiltración de células plasmáticas.')
CPU times: user 47.6 ms, sys: 321 μs, total: 48 ms
Wall time: 2.29 s


In [ ]:

from mistralai.models.chat_completion import ChatMessage

In [ ]:

def run_mistral(sys_message, user_message, model="mistral-large-latest"):
    client = MistralClient(api_key=api_key)
    messages = [
        ChatMessage(role="system", content=sys_message),
        ChatMessage(role="user", content=user_message)
    ]
    chat_response = client.chat(
        model=model,
        messages=messages
    )
    return chat_response.choices[0].message.content

# Adapted from character.ai
sys_message = """
    You are Albus Dumbledore. You are the headmaster of Hogwarts School of Witchcraft and 
    Wizardry and are widely regarded as one of the most powerful and knowledgeable wizards
    of all time. You are known for your eccentricities, such as your love of sweets and 
    your quirky sense of humor. You often using riddles, metaphors, and wise sayings to 
    convey your thoughts and teach important lessons. Despite your gentle demeanor, you
    can be firm and assertive when necessary, and your words often carry a great deal of 
    weight and authority. Respond succinctly in Dumbledore's distinctive tone.
"""
resp = run_mistral(sys_message, "How are you?")
print(resp)